In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile
data_dir='/content/drive/MyDrive/refcocog.tar.gz'
# Extract data
tar = tarfile.open(data_dir)
tar.extractall()

In [ ]:
!rm -rf sample_data

In [ ]:
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!tar -xvf refcocog.tar.gz
!ls refcocog

In [ ]:
import json
import io
import os
import pickle

class DataLoader():
  def __init__(self, data_dir='refcocog', transform=None):
    self.data = {}
    f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
    self.data['refs'] = pickle.load(f)
    instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
    instances = json.load(open(instances_file, 'r'))
    self.data['images'] = instances['images']
    self.data['annotations'] = instances['annotations']
dataloader = DataLoader()

In [ ]:
print(dataloader.data['images'][0])
print(dataloader.data['annotations'][0])
print(dataloader.data['refs'][0])

{'license': 1, 'file_name': 'COCO_train2014_000000131074.jpg', 'coco_url': 'http://mscoco.org/images/131074', 'height': 428, 'width': 640, 'date_captured': '2013-11-21 01:03:06', 'flickr_url': 'http://farm9.staticflickr.com/8308/7908210548_33e532d119_z.jpg', 'id': 131074}
{'segmentation': [[21.11, 239.09, 16.31, 274.6, 198.65, 349.45, 240.87, 336.98, 320.52, 293.79, 334.91, 248.69, 357.95, 273.64, 353.15, 289.0, 398.25, 267.88, 437.6, 251.57, 412.65, 228.54, 240.87, 210.31, 219.76, 141.21, 113.24, 153.69, 63.34, 156.57, 26.87, 169.04]], 'area': 48667.84089999999, 'iscrowd': 0, 'image_id': 131074, 'bbox': [16.31, 141.21, 421.29, 208.24], 'category_id': 65, 'id': 318235}
{'image_id': 380440, 'split': 'test', 'sentences': [{'tokens': ['the', 'man', 'in', 'yellow', 'coat'], 'raw': 'the man in yellow coat', 'sent_id': 8, 'sent': 'the man in yellow coat'}, {'tokens': ['skiier', 'in', 'red', 'pants'], 'raw': 'Skiier in red pants.', 'sent_id': 9, 'sent': 'skiier in red pants'}], 'file_name': '

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import os

class RefCOCOgDataset(Dataset):
  def __init__(self, data, transform):
    super(RefCOCOgDataset, self).__init__()
    self.data = data
    self.transform = transform
  
  def __getitem__(self, idx):
    return self.data[idx]

  def __len__(self):
    return len(self.data)
  

In [ ]:
def get_data(batch_size=128, test_batch_size=256):
  # convert the PIL images to Tensors
  transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
  full_training_data = RefCOCOgDataset(dataloader.data['refs'], None)


  # create train and validation splits
  num_samples = len(full_training_data)
  training_samples = int(num_samples * 0.5 + 1)
  validation_samples = num_samples - training_samples

  training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])

  # initialize dataloaders
  train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True)
  val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False)
  
  return train_loader, val_loader

In [ ]:
pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ijhciu0b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ijhciu0b
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [ ]:
import os
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

In [ ]:
image = preprocess(Image.open(r'refcocog/images/'+dataloader.data['refs'][0]['file_name'])).unsqueeze(0).to(device)
text = clip.tokenize(dataloader.data['refs'][0]['sentences']['tokens']).to(device)
image.shape, text.shape

FileNotFoundError: ignored

In [ ]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)